In [ ]:
import cv2

# Load the SSD deep learning model
model_path = "frozen_inference_graph.pb"
config_path = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
net = cv2.dnn_DetectionModel(model_path, config_path)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

# Load the video
video_path = "hockey_game.mp4"
cap = cv2.VideoCapture(video_path)

# Create the tracker object
tracker = cv2.TrackerKCF_create()

# Loop through each frame of the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect players in the current frame
    class_ids, confidences, boxes = net.detect(frame, confThreshold=0.5, nmsThreshold=0.4)

    # Track players using the KCF tracker
    for box in boxes:
        x, y, w, h = box
        tracker.init(frame, (x, y, w, h))
        ok, bbox = tracker.update(frame)
        if ok:
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), (0, 255, 0), 2)

    # Show the current frame with player boxes and tracking results
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video and close the windows
cap.release()
cv2.destroyAllWindows()